# Get data DMI data from API

In [ ]:
import json
import requests
import pandas as pd
from datetime import datetime

APIKEY = ''
PARAMS = ['wind_dir_past1h', 'wind_speed_past1h', 'precip_past1h', 'humidity_past1h']

for PARAM in PARAMS:
    START_DATE = datetime.strptime('2018/11/05', '%Y/%m/%d').strftime('%Y-%m-%dT%H:%M:%SZ')
    END_DATE = datetime.strptime('2020/03/12', '%Y/%m/%d').strftime('%Y-%m-%dT%H:%M:%SZ')

    url = 'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items'
    payload = {
        'api-key': APIKEY,
        'limit': 100000,
        'stationId': '06181',
        'datetime': f'{START_DATE}/{END_DATE}',
        'parameterId': PARAM,
    }
    r = requests.get(url, params=payload)
    response_dct = r.json()
    print(len(response_dct['features']))

    with open(f'{PARAM}.json', 'w') as f:
        json.dump(response_dct, f)

# Upload to a dataframe

In [ ]:
big_df = pd.DataFrame()
for param in PARAMS:
    with open(f'{param}.json', 'r') as f:
        dct = json.load(f)
    if not dct['features']:
        continue
    df = pd.json_normalize(dct['features'])
    df['geometry.coordinates'] = df['geometry.coordinates'].apply(lambda lst: [str(x) for x in lst]).str.join(',')
    df.to_csv(f'{param}.csv', index=False)
    big_df = big_df.append(df)

# Visualize

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.facecolor'] = 'white'
big_df['properties.observed'] = pd.to_datetime(df['properties.observed'])

fig = plt.figure(figsize=(10, 7))
for i, (name, g) in enumerate(big_df.groupby('properties.parameterId')):  
    ax = fig.add_subplot(2,2,i+1)
    g.set_index('properties.observed').groupby(pd.Grouper(freq='W'))['properties.value'].mean().dropna().plot(ax=ax)
    ax.set_xlabel(name)
plt.tight_layout()
plt.savefig('plots.png')

In [ ]:
for i, (name, g) in enumerate(big_df.groupby('properties.parameterId')):  
    print(name, g['properties.observed'].min())

In [ ]:
df = pd.DataFrame({'local_time': ['2019-06-18 23:07:52', '2019-06-18 23:07:53'], 'UFP': [27800, 27100]})
df.local_time = pd.to_datetime(df.local_time).dt.floor('h').dt.strftime('%Y-%m-%dT%H:%M:%SZ')
df